In [1]:
from langchain import PromptTemplate

template = """Question: {question}

Answer: """
prompt = PromptTemplate(
        template=template,
    input_variables=['question']
)

# user question
question = "Which NFL team won the Super Bowl in the 2010 season?"


In [2]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [3]:
# slow, really really slow.

from langchain import HuggingFaceHub, LLMChain

# initialize Hub LLM
hub_llm = HuggingFaceHub(
    repo_id='google/flan-t5-xl',
    model_kwargs={'temperature':1e-10}
)

# create prompt template > LLM chain
llm_chain = LLMChain(
    prompt=prompt,
    llm=hub_llm
)

# ask the user question about NFL 2010
print(llm_chain.run(question))

c:\develop\github\ai-playground\src\python\langchainVarious\langchain\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ValueError: Error raised by inference API: Model google/flan-t5-xl time out

The code above is a Python script that demonstrates how to use the `langchain` package to generate language model prompts using a pre-trained model from the Hugging Face model hub. The script imports two classes from the `langchain` package: `HuggingFaceHub` and `LLMChain`.

The `HuggingFaceHub` class is used to load a pre-trained language model from the Hugging Face model hub. In this case, the `repo_id` parameter is set to `'google/flan-t5-xl'`, which loads a pre-trained T5 model from Google. The `model_kwargs` parameter is used to set the temperature of the model to `1e-10`, which makes the model output deterministic.

The `LLMChain` class is used to create a prompt template for the language model. The `prompt` parameter is set to a string that contains the initial prompt for the language model. The `llm` parameter is set to the `hub_llm` object, which is the pre-trained T5 model loaded from the Hugging Face model hub.

The `run` method of the `LLMChain` class is used to generate a prompt from the language model. The `question` parameter is set to a string that contains a question about NFL 2010. The `run` method generates a prompt by appending the `question` string to the `prompt` string and passing the resulting string to the pre-trained T5 model. The generated prompt is then returned as a string.

Overall, this code demonstrates how to use the `langchain` package to generate language model prompts using a pre-trained model from the Hugging Face model hub. The `HuggingFaceHub` class is used to load the pre-trained model, and the `LLMChain` class is used to create a prompt template and generate prompts from the model. The generated prompt can be used for a variety of natural language processing tasks, such as question answering, text completion, and text generation.

In [7]:
from langchain.llms import AzureOpenAI
from langchain import HuggingFaceHub, LLMChain
import openai

# Remember that the deployment name can be different from the model name
# and that you need to create a .env with the OPENAI_API_KEY variable
#and OPENAI_API_BASE
llm = AzureOpenAI(
    openai_api_version="2023-03-15-preview",
    deployment_name="text-davinci-003", 
    model_name="text-davinci-003"
)

llm_chain = LLMChain(
    prompt=prompt,
    llm=llm
)

print(llm_chain.run(question))

 The Green Bay Packers won the Super Bowl in the 2010 season.


We can do multiple question in a single request, this will perform multiple query to the model.

In [8]:
from pprint import pprint
qs = [
    {'question': "Which NFL team won the Super Bowl in the 2010 season?"},
    {'question': "If I am 6 ft 4 inches, how tall am I in centimeters?"},
    {'question': "Who was the 12th person on the moon?"},
    {'question': "How many eyes does a blade of grass have?"}
]
result = llm_chain.generate(qs)
pprint(result, width=80)

LLMResult(generations=[[Generation(text=' The Green Bay Packers won the Super Bowl in the 2010 season.', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text=' 193.04 centimeters', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text=' The 12th person on the moon was Astronaut Alan Bean, part of the Apollo 12 mission.', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text=' A blade of grass does not have eyes.', generation_info={'finish_reason': 'stop', 'logprobs': None})]], llm_output={'token_usage': {'total_tokens': 121, 'completion_tokens': 46, 'prompt_tokens': 75}, 'model_name': 'text-davinci-003'}, run=RunInfo(run_id=UUID('1e7ea884-f1c6-450a-9297-3b59056d790c')))


Now feed all the question all at once

In [10]:
multi_template = """Answer the following questions one at a time.

Questions:
{questions}

Answers:
"""
long_prompt = PromptTemplate(template=multi_template, input_variables=["questions"])

llm_chain = LLMChain(
    prompt=long_prompt,
    llm=llm
)

qs_str = (
    "Which NFL team won the Super Bowl in the 2010 season?\n" +
    "If I am 6 ft 4 inches, how tall am I in centimeters?\n" +
    "Who was the 12th person on the moon?" +
    "How many eyes does a blade of grass have?"
)

pprint(llm_chain.run(qs_str))

('The New Orleans Saints won the Super Bowl in the 2010 season.\n'
 'If you are 6 ft 4 inches, you are 193.04 centimeters tall.\n'
 'The 12th person on the moon was Harrison Schmitt.\n'
 'A blade of grass does not have any eyes.')
